In [2]:
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
# import matplotlib
import os
# matplotlib.use('TkAgg')
# import matplotlib.pyplot as plt

def pHash(img):
    # 感知哈希算法
    # 缩放32*32
    img = cv2.resize(img, (32, 32))   # , interpolation=cv2.INTER_CUBIC
 
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 将灰度图转为浮点型，再进行dct变换
    dct = cv2.dct(np.float32(gray))
    # opencv实现的掩码操作
    dct_roi = dct[0:8, 0:8]
 
    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash

def cmpHash(hash1, hash2):
    # Hash值对比
    # 算法中1和0顺序组合起来的即是图片的指纹hash。顺序不固定，但是比较的时候必须是相同的顺序。
    # 对比两幅图的指纹，计算汉明距离，即两个64位的hash值有多少是不一样的，不同的位数越小，图片越相似
    # 汉明距离：一组二进制数据变成另一组数据所需要的步骤，可以衡量两图的差异，汉明距离越小，则相似度越高。汉明距离为0，即两张图片完全一样
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1) != len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 不相等则n计数+1，n最终为相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n
 
def getImageByUrl(url):
    # 根据图片url 获取图片对象
    html = requests.get(url, verify=False)
    image = Image.open(BytesIO(html.content))
    return image

def runAllImageSimilaryFun(para1, para2):
    # 均值、差值、感知哈希算法三种算法值越小，则越相似,相同图片值为0
    # 三直方图算法和单通道的直方图 0-1之间，值越大，越相似。 相同图片为1
 
    # t1,t2   14;19;10;  0.70;0.75
    # t1,t3   39 33 18   0.58 0.49
    # s1,s2  7 23 11     0.83 0.86  挺相似的图片
    # c1,c2  11 29 17    0.30 0.31
 
    if para1.startswith("http"):
         # 根据链接下载图片，并转换为opencv格式
        img1 = getImageByUrl(para1)
        img1 = cv2.cvtColor(np.asarray(img1), cv2.COLOR_RGB2BGR)
 
        img2 = getImageByUrl(para2)
        img2 = cv2.cvtColor(np.asarray(img2), cv2.COLOR_RGB2BGR)
    else:
        # 通过imread方法直接读取物理路径
        img1 = cv2.imread(para1)
        img2 = cv2.imread(para2)
 
 
    hash1 = pHash(img1)
    hash2 = pHash(img2)
    n3 = cmpHash(hash1, hash2)
    return n3
#     if n3 >= 10:
#         print('感知哈希算法相似度pHash：', n3)

In [3]:
path = "D:/image/dataset/image_test1/hg/"
file_name = os.listdir(path)

item_list =[] 
for item in file_name:
    if item[-4:].lower() == '.jpg' or item[-4:].lower() == '.png' or item[-5:].lower() == '.jpeg':
        item = path + item
        item_list.append(item)

        
list_set = []
for i in range(len(item_list)):
    for j in range(i+1,len(item_list)):
        list = [item_list[i],item_list[j]]
        list_set.append(list)        

if __name__ == "__main__":
    for i in range(len(list_set)):
        p1 = list_set[i][0]
        p2 = list_set[i][1]
        n3 = runAllImageSimilaryFun(p1,p2)
        if n3 >= 10:
            print(p1,p2)

D:/image/dataset/image_test1/hg/hg. (1).jpg D:/image/dataset/image_test1/hg/hg. (12).jpg
D:/image/dataset/image_test1/hg/hg. (1).jpg D:/image/dataset/image_test1/hg/hg. (6).png
D:/image/dataset/image_test1/hg/hg. (1).jpg D:/image/dataset/image_test1/hg/hg. (61).jpg
D:/image/dataset/image_test1/hg/hg. (10).jpg D:/image/dataset/image_test1/hg/hg. (68).jpg
D:/image/dataset/image_test1/hg/hg. (11).jpg D:/image/dataset/image_test1/hg/hg. (61).jpg
D:/image/dataset/image_test1/hg/hg. (12).jpg D:/image/dataset/image_test1/hg/hg. (61).jpg
D:/image/dataset/image_test1/hg/hg. (12).jpg D:/image/dataset/image_test1/hg/hg. (68).jpg
D:/image/dataset/image_test1/hg/hg. (13).jpg D:/image/dataset/image_test1/hg/hg. (61).jpg
D:/image/dataset/image_test1/hg/hg. (13).jpg D:/image/dataset/image_test1/hg/hg. (68).jpg
D:/image/dataset/image_test1/hg/hg. (5).jpg D:/image/dataset/image_test1/hg/hg. (61).jpg
D:/image/dataset/image_test1/hg/hg. (5).jpg D:/image/dataset/image_test1/hg/hg. (68).jpg
D:/image/dataset

In [52]:
if __name__ == "__main__":
    p1="D:/image/dataset/image_test1/hg/hg. (1).jpg"
    p2="D:/image/dataset/image_test1/hg/hg. (5).jpg"
    runAllImageSimilaryFun(p1,p2)

感知哈希算法相似度pHash： 9
